In [60]:
# MNIST dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

# loader
from torch.utils.data import DataLoader, ConcatDataset, Dataset

# neural network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# plotting
import matplotlib.pyplot as plt

# numpy
import numpy as np

# for deep copy
import copy

In [61]:
# create dataset class
class CTDataset(Dataset):
    def __init__(self, filepath):
        self.x, self.y = torch.load(filepath)
        self.x = self.x.view(-1, 1, 28, 28)
        self.x = self.x / 255.0

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [62]:
# load and concat datasets
mnist_train = CTDataset('../data/MNIST/processed/training.pt')
mnist_test = CTDataset('../data/MNIST/processed/test.pt')
digits_train = CTDataset('../data/personal/digits_training.pt')
digits_test = CTDataset('../data/personal/digits_test.pt')

train_ds = ConcatDataset([mnist_train, digits_train])
test_ds = ConcatDataset([mnist_test, digits_test])

In [63]:
type(train_ds)

torch.utils.data.dataset.ConcatDataset

In [64]:
# see if numbers are ok
def visualize_data(data):
    img = data[0].numpy()
    label = data[1].argmax()
    plt.imshow(img, cmap='gray')
    plt.title(f'Label: {label}')
    plt.show()

In [65]:
loaders = {
    'train': DataLoader(train_ds, batch_size=100, shuffle=True),
    'test': DataLoader(test_ds, batch_size=100, shuffle=False)
}

In [66]:
# Define model
class CNN_Ultra(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN_Ultra, self).__init__()

        # Layer 1: Learn basic shapes (curves, lines)
        # Input: (1, 28, 28) -> Output: (32, 28, 28)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Pool reduces to (32, 14, 14)

        # Layer 2: Learn combinations of shapes (loops, corners)
        # Input: (32, 14, 14) -> Output: (64, 14, 14)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        # Pool reduces to (64, 7, 7)

        # Fully Connected Layers
        # We flattened 64 channels * 7 * 7 pixels = 3136
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.dropout = nn.Dropout(0.5) # Crucial to prevent overfitting on the 8 fonts
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)

        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)

        # Flatten
        x = x.view(x.size(0), -1) # Flatten (Batch_Size, 3136)

        # Classifier
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [67]:
# Instantiate and define training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN_Ultra().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()   # put model in training mode
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()   # initialize with 0
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loaders['train'].dataset),
                100. * batch_idx / len(loaders['train']), loss.item()))

def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')
    return test_loss

In [68]:
# train
def train_until_best(epochs, patience=5):
    best_loss = float('inf')
    patience_counter = 0
    best_model_wts = None

    for epoch in range(1, epochs + 1):
        train(epoch)
        test_loss = test()

        if test_loss < best_loss:
            best_loss = test_loss
            patience_counter = 0
            best_model_wts = copy.deepcopy(model.state_dict())
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)
        torch.save(model.state_dict(), '../models/CNN_Ultra.pth')
        print('Best model saved!')

    return model

model = train_until_best(100, patience=5)

Train Epoch: 1 [0/132000 (0%)]	Loss: 2.408971
Train Epoch: 1 [10000/132000 (8%)]	Loss: 0.142575
Train Epoch: 1 [20000/132000 (15%)]	Loss: 0.109464
Train Epoch: 1 [30000/132000 (23%)]	Loss: 0.066225
Train Epoch: 1 [40000/132000 (30%)]	Loss: 0.095007
Train Epoch: 1 [50000/132000 (38%)]	Loss: 0.061703
Train Epoch: 1 [60000/132000 (45%)]	Loss: 0.139804
Train Epoch: 1 [70000/132000 (53%)]	Loss: 0.082304
Train Epoch: 1 [80000/132000 (61%)]	Loss: 0.078789
Train Epoch: 1 [90000/132000 (68%)]	Loss: 0.093572
Train Epoch: 1 [100000/132000 (76%)]	Loss: 0.098052
Train Epoch: 1 [110000/132000 (83%)]	Loss: 0.037972
Train Epoch: 1 [120000/132000 (91%)]	Loss: 0.063318
Train Epoch: 1 [130000/132000 (98%)]	Loss: 0.017643

Test set: Average loss: 0.0003, Accuracy: 17819/18000 (99%)

Train Epoch: 2 [0/132000 (0%)]	Loss: 0.033132
Train Epoch: 2 [10000/132000 (8%)]	Loss: 0.029177
Train Epoch: 2 [20000/132000 (15%)]	Loss: 0.042768
Train Epoch: 2 [30000/132000 (23%)]	Loss: 0.022056
Train Epoch: 2 [40000/132000